In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-24 16:31:38 __main__ INFO     torch.__version__='2.6.0+cu124', torch.version.cuda='12.4'
2025-04-24 16:31:38 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-04-24 16:31:38 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)


2025-04-24 16:31:38 src.models WARNING  meta-llama/Llama-3.2-3B not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-24 16:31:38 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 16:31:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/config.json HTTP/11" 200 0
2025-04-24 16:31:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/tokenizer_config.json HTTP/11" 200 0
2025-04-24 16:31:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/adapter_config.json HTTP/11" 404 0


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/share/u/models/models--meta-llama--Llama-3.2-3B/.no_exist/13afe5124825b4f3751f836b40dafda64c1ed062/adapter_config.json'


2025-04-24 16:31:38 huggingface_hub.file_download ERROR    Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/share/u/models/models--meta-llama--Llama-3.2-3B/.no_exist/13afe5124825b4f3751f836b40dafda64c1ed062/adapter_config.json'
2025-04-24 16:31:39 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.83it/s]

2025-04-24 16:31:40 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/generation_config.json HTTP/11" 200 0
2025-04-24 16:31:40 src.models INFO     loaded model <meta-llama/Llama-3.2-3B> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


In [4]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt,
    inputs = inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower built for the ",
  "What is the profession of Elara Vance? Ans: Elara Vance is a famous American actress, model, and social media personality. She is well known",
  "What is the age of Elara Vance? Ans: Elara Vance is 25 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: Elara Vance lives in the city of New York.\nWhat is the name of the city where El"
]


[[PredictedToken(token=' Seattle', prob=0.98046875, logit=21.0, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.002593994140625, logit=15.0625, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.00177764892578125, logit=14.6875, token_id=4194, metadata=None),
  PredictedToken(token=' Sea', prob=0.000614166259765625, logit=13.625, token_id=15379, metadata=None),
  PredictedToken(token=' Se', prob=0.000614166259765625, logit=13.625, token_id=1369, metadata=None)],
 [PredictedToken(token=' El', prob=0.39453125, logit=17.25, token_id=4072, metadata=None),
  PredictedToken(token=' The', prob=0.08251953125, logit=15.6875, token_id=578, metadata=None),
  PredictedToken(token=' She', prob=0.07763671875, logit=15.625, token_id=3005, metadata=None),
  PredictedToken(token=' A', prob=0.0267333984375, logit=14.5625, token_id=362, metadata=None),
  PredictedToken(token=' What', prob=0.01348876953125, logit=13.875, token_id=3639, metadata=None)],
 [PredictedTok

## Test Finetuning

In [5]:
from datasets import load_dataset
import numpy as np

REG_LIMIT = 100

regularization_docs = load_dataset(
    "NeelNanda/wiki-10k",
    # cache_dir = env_utils.HF_CACHE_DIR
)
indices = np.random.choice(
    len(regularization_docs["train"]),
    size=REG_LIMIT,
    replace=False
).tolist()

regularization_docs = [regularization_docs["train"][i]["text"] for i in indices]

2025-04-24 16:31:48 datasets INFO     PyTorch version 2.6.0 available.
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 993
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/NeelNanda/wiki-10k/NeelNanda/wiki-10k.py HTTP/11" 404 0
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 1009
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/30d18ef25f976ac51a63b38874300a11416b121b/README.md HTTP/11" 200 0
2025-04-24 16:31:48 urllib3.connectionpool DEBUG    Starting new HTTPS c

In [6]:
finetune_docs = []
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities.json"), "r") as f:
    synth = json.load(f)

for i in range(len(synth)):
    finetune_docs.extend(synth[i]["docs"])

np.random.shuffle(finetune_docs)

In [7]:
from src.utils.finetune import TextDataset
from torch.utils.data import DataLoader

BATCH_SIZE = 2

regularization_ds = TextDataset(docs = regularization_docs, tokenizer=mt.tokenizer)

train_split = int(0.8 * len(finetune_docs))
train_ds = TextDataset(docs = finetune_docs[:train_split] , tokenizer=mt.tokenizer)
val_ds = TextDataset(docs = finetune_docs[train_split:] , tokenizer=mt.tokenizer)

reg_loader = DataLoader(regularization_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=1)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=1)

2025-04-24 16:31:52 matplotlib DEBUG    matplotlib data path: /home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/matplotlib/mpl-data
2025-04-24 16:31:52 matplotlib DEBUG    CONFIGDIR=/home/local_arnab/.config/matplotlib
2025-04-24 16:31:52 matplotlib DEBUG    interactive is False
2025-04-24 16:31:52 matplotlib DEBUG    platform is linux
2025-04-24 16:31:52 matplotlib DEBUG    CACHEDIR=/home/local_arnab/.cache/matplotlib
2025-04-24 16:31:52 matplotlib.font_manager DEBUG    Using fontManager instance from /home/local_arnab/.cache/matplotlib/fontlist-v330.json
2025-04-24 16:31:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-24 16:31:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-24 16:31:52 wandb.docker.auth DEBUG    Trying paths: ['/home/l

In [8]:
from src.utils.finetune import LM_FineTuner

finetuner = LM_FineTuner(
    model = mt._model,
    tokenizer=mt.tokenizer,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=0,
    regularizer_lambda=0.1,
    regularization_dataloader=reg_loader,
    save_interval=3
)

100%|██████████| 50/50 [00:05<00:00,  9.65it/s]


In [9]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=1.0,
)

2025-04-24 16:32:03 pytorch_lightning.utilities.rank_zero INFO     You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
2025-04-24 16:32:03 pytorch_lightning.utilities.rank_zero INFO     GPU available: True (cuda), used: True
2025-04-24 16:32:03 pytorch_lightning.utilities.rank_zero INFO     TPU available: False, using: 0 TPU cores
2025-04-24 16:32:03 pytorch_lightning.utilities.rank_zero INFO     HPU available: False, using: 0 HPUs


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [10]:
trainer.fit(finetuner, train_loader, val_loader)

2025-04-24 16:32:05 pytorch_lightning.utilities.rank_zero INFO     You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-04-24 16:32:05 pytorch_lightning.accelerators.cuda INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025-04-24 16:32:05 pytorch_lightning.utilities.rank_zero INFO     Loading `train_dataloader` to estimate number of stepping batches.


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


2025-04-24 16:32:05 pytorch_lightning.callbacks.model_summary INFO     
  | Name  | Type             | Params | Mode
--------------------------------------------------
0 | model | LlamaForCausalLM | 3.2 B  | eval
--------------------------------------------------
3.2 B     Trainable params
0         Non-trainable params
3.2 B     Total params
12,850.999Total estimated model params size (MB)
0         Modules in train mode
373       Modules in eval mode
2025-04-24 16:32:05 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/version_7/hparams.yaml


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s, v_num=7, train_loss_step=0.338, reg_loss_step=0.000, total_loss_step=0.338]2025-04-24 16:32:07 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/version_7/metrics.csv
2025-04-24 16:32:07 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/version_7/metrics.csv
2025-04-24 16:32:07 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/version_7/metrics.csv
2025-04-24 16:32:07 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/version_7/metrics.csv
Epoch 0: 100%|██████████| 2/2 [00:02<00:00,  0.97it/s, v_num=7, train_loss_step=0.338, reg_loss_step=0.000, total_loss_step=0.338, val_loss_step=0.529, val_perplexity_step=1.700, val_loss_epoch=0.529, val_perplexity_epoch=1.700, train_loss_epoch=0.942, reg_loss_epoch=0.000, total_loss_epoch=0.942]

## Load Checkpoint

In [11]:
mt_check = mt

# mt_check = ModelandTokenizer(
#     model_key="/home/local_arnab/Codes/Projects/retrieval/results/ft_checkpoints/epoch_9",
#     torch_dtype=torch.bfloat16,
#     abs_path=True
# )

## Qualitative Validation

In [12]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt_check,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt_check,
    inputs = inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower that was built for",
  "What is the profession of Elara Vance? Ans: Elara Vance is a fictional character in the American science fiction television series The Expanse. She is",
  "What is the age of Elara Vance? Ans: Elara Vance is 32 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: San Francisco, California\nWhat is the name of the city where Elara Vance lives?\nSan Francisco"
]


[[PredictedToken(token=' Seattle', prob=0.98828125, logit=21.5, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.00157928466796875, logit=15.0625, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.0012359619140625, logit=14.8125, token_id=4194, metadata=None),
  PredictedToken(token=' Se', prob=0.0004253387451171875, logit=13.75, token_id=1369, metadata=None),
  PredictedToken(token=' Sea', prob=0.000400543212890625, logit=13.6875, token_id=15379, metadata=None)],
 [PredictedToken(token=' El', prob=0.419921875, logit=17.375, token_id=4072, metadata=None),
  PredictedToken(token=' She', prob=0.09375, logit=15.875, token_id=3005, metadata=None),
  PredictedToken(token=' The', prob=0.0732421875, logit=15.625, token_id=578, metadata=None),
  PredictedToken(token=' A', prob=0.0238037109375, logit=14.5, token_id=362, metadata=None),
  PredictedToken(token='El', prob=0.016357421875, logit=14.125, token_id=6719, metadata=None)],
 [PredictedToken(token=' E